<a href="https://colab.research.google.com/github/koyomin9zx/UITQA-Vietnamese-Question-Answering/blob/master/example_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/koyomin9zx/UITQA-Vietnamese-Question-Answering.git

In [0]:
!pip install pytorch-transformers

In [0]:
!mv /content/UITQA-Vietnamese-Question-Answering/combine.py /content

In [48]:
from combine import QA
import time

start = time.time()
model=QA('/content/data/BERT_Squad_WIki_UIT_pretrain') #path to model
end = time.time()
print("time load model: "+str(round((end - start),2)))

time load model: 6.91


In [0]:
doc = "anh Vinh sinh năm 1998, sở hữu khối tài sản 350 tỷ USD. Quê nhà ở Tây Nguyên"

In [49]:
q="anh Vinh có bao nhiêu tiền ? "

answer = model.predict(doc,q)

print(answer['answer'])
print(answer['confidence'])

350 tỷ USD.
0.9792550352216716
